In [3]:


import os
os.environ["DASHSCOPE_API_KEY"] = os.getenv("DASHSCOPE_API_KEY")  # 阿里云
# os.getenv("DASHSCOPE_API_KEY")

In [1]:
import os
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings.dashscope import DashScopeEmbeddings

In [2]:
# 使用稳定可靠的API服务
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", 
    dashscope_api_key= os.getenv("DASHSCOPE_API_KEY")
)
#测试词嵌入
text = "This is a test document."
query_result = embeddings.embed_query(text)
print(query_result)

[0.3107011616230011, 0.9425862431526184, -0.9483913779258728, 2.639361619949341, -2.1982150077819824, -0.12639787793159485, 1.0644768476486206, -1.463064432144165, 0.4155646562576294, 0.9022487998008728, -0.9056667685508728, 0.2552490234375, 1.5458712577819824, -0.2592027485370636, 1.7698636054992676, 0.1977505087852478, -0.6691555380821228, -3.3732097148895264, 0.8468695878982544, 0.5063815712928772, 1.02392578125, 1.0923393964767456, 0.9745347499847412, 0.0087415911257267, -0.3915913999080658, -0.1623435616493225, -0.9985283613204956, -0.5642293095588684, -1.1784464120864868, 0.10965368151664734, -0.8466660976409912, -0.1585591584444046, -0.2599538266658783, 2.209174156188965, 2.269822835922241, -0.2519938051700592, 0.0421498604118824, -0.6886189579963684, 1.639203429222107, 0.1796434223651886, 2.9693570137023926, 1.4052726030349731, 4.72607421875, 0.1977708637714386, 0.1952701210975647, -0.9378458857536316, 0.7993571162223816, 0.16235817968845367, -2.1394450664520264, -1.26909720897

In [3]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
#创建一个VectorstoreIndexCreator实例，
#通过vectorstore_cls参数指定使用DocArrayInMemorySearch作为向量数据库，
#该数据库将向量存储在内存中。
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])

/Users/zhaoyang/opt/anaconda3/lib/python3.9/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
query ="Please list all your shirts with sun protection \
in a table in markdown and summarize each one."

In [6]:
llm = ChatOpenAI(
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_name="qwen-plus",	# 模型名称
    temperature=0.9
)

/var/folders/2p/9b5w6pj56_s1vvr_y6r4cmqc0000gn/T/ipykernel_40332/279262759.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [7]:
response = index.query(query, llm=llm)  

In [8]:
display(Markdown(response))

| Item # | Name | Summary |
|--------|------|---------|
| 255    | Sun Shield Shirt | A high-performance sun shirt with a slightly fitted design that falls at the hip. Made from 78% nylon and 22% Lycra, it offers UPF 50+ protection, is moisture-wicking, and abrasion-resistant. Recommended by The Skin Cancer Foundation for UV protection. Handwash and line dry. |
| 535    | Men's TropicVibe Shirt, Short-Sleeve | A relaxed fit men's shirt made from 71% Nylon and 29% Polyester with UPF 50+ protection. It features wrinkle resistance, front and back cape venting for cooling, two front bellows pockets, and machine washable care. Provides SPF 50+ sun protection blocking 98% of harmful rays. |
| 618    | Men's Tropical Plaid Short-Sleeve Shirt | A lightweight hot-weather shirt made of 100% polyester with UPF 50+ protection. It has a traditional fit, wrinkle resistance, front and back cape venting for breathability, and two front bellows pockets. Offers SPF 50+ sun protection blocking 98% of harmful rays. |
| 293    | Girls' Ocean Breeze Long-Sleeve Stripe Shirt | A long-sleeve rash guard for girls made from a nylon and Lycra-elastane blend with UPF 50+ protection. It is quick-drying, fade-resistant, seawater-resistant, and coordinates easily with swimsuits. Machine washable and recommended by The Skin Cancer Foundation for UV protection. Blocks 98% of harmful rays. |

In [9]:
loader = CSVLoader(file_path=file)

In [10]:
docs = loader.load()

In [11]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 0}, page_content=": 0\nname: Women's Campside Oxfords\ndescription: This ultracomfortable lace-to-toe Oxford boasts a super-soft canvas, thick cushioning, and quality construction for a broken-in feel from the first time you put them on. \n\nSize & Fit: Order regular shoe size. For half sizes not offered, order up to next whole size. \n\nSpecs: Approx. weight: 1 lb.1 oz. per pair. \n\nConstruction: Soft canvas material for a broken-in feel and look. Comfortable EVA innersole with Cleansport NXT® antimicrobial odor control. Vintage hunt, fish and camping motif on innersole. Moderate arch contour of innersole. EVA foam midsole for cushioning and support. Chain-tread-inspired molded rubber outsole with modified chain-tread pattern. Imported. \n\nQuestions? Please contact us for any inquiries.")

In [12]:
#测试词嵌入
text = "Hi my name is Harrison"
embed = embeddings.embed_query(text)
print(len(embed))
print(embed[:5])


1536
[0.585662841796875, 6.017246723175049, 0.7936912178993225, 0.0716334730386734, -0.3416312038898468]


In [13]:
db = DocArrayInMemorySearch.from_documents(
    docs, 
    embeddings
)

In [14]:
query = "Please suggest a shirt with sunblocking"

In [15]:
docs = db.similarity_search(query)

In [16]:
len(docs)

4

In [17]:
docs[0]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 255}, page_content=': 255\nname: Sun Shield Shirt by\ndescription: "Block the sun, not the fun – our high-performance sun shirt is guaranteed to protect from harmful UV rays. \n\nSize & Fit: Slightly Fitted: Softly shapes the body. Falls at hip.\n\nFabric & Care: 78% nylon, 22% Lycra Xtra Life fiber. UPF 50+ rated – the highest rated sun protection possible. Handwash, line dry.\n\nAdditional Features: Wicks moisture for quick-drying comfort. Fits comfortably over your favorite swimsuit. Abrasion resistant for season after season of wear. Imported.\n\nSun Protection That Won\'t Wear Off\nOur high-performance fabric provides SPF 50+ sun protection, blocking 98% of the sun\'s harmful rays. This fabric is recommended by The Skin Cancer Foundation as an effective UV protectant.')

In [18]:
retriever = db.as_retriever()#检索器

In [19]:
llm = ChatOpenAI(
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_name="qwen-plus",	# 模型名称
    temperature=0.9
)

In [20]:
qdocs = "".join([docs[i].page_content for i in range(len(docs))])

In [20]:
#response = llm.call_as_llm(f"{qdocs} Question: Please list all your \
# shirts with sun protection in a table in markdown and summarize each one.") 

In [21]:
response = llm.invoke(f"{qdocs} Question: Please list all your \
shirts with sun protection in a table in markdown and summarize each one.")

In [24]:
# display(Markdown(response["query"]))
display(Markdown(response))

TypeError: Markdown expects text, not AIMessage(content="Below is a table in markdown format summarizing all the shirts with sun protection:\n\n```markdown\n| Name                          | Description Summary                                                                                                                                     |\n|-------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt by           | A high-performance, slightly fitted sun shirt made of 78% nylon and 22% Lycra Xtra Life fiber. Offers UPF 50+ protection, moisture-wicking, and abrasion resistance. Handwash recommended. |\n| Men's TropicVibe Shirt        | A relaxed-fit short-sleeve shirt with UPF 50+ sun protection. Made from a lightweight nylon-polyester blend, it features wrinkle-resistant fabric, front and back cape venting, and two bellows pockets. Machine 

In [25]:
display(response)

AIMessage(content="Below is a table in markdown format summarizing all the shirts with sun protection:\n\n```markdown\n| Name                          | Description Summary                                                                                                                                     |\n|-------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt by           | A high-performance, slightly fitted sun shirt made of 78% nylon and 22% Lycra Xtra Life fiber. Offers UPF 50+ protection, moisture-wicking, and abrasion resistance. Handwash recommended. |\n| Men's TropicVibe Shirt        | A relaxed-fit short-sleeve shirt with UPF 50+ sun protection. Made from a lightweight nylon-polyester blend, it features wrinkle-resistant fabric, front and back cape venting, and two bellows pockets. Machine washable. |\n| Tropical Breeze Shirt  

In [26]:
# display(Markdown(response["result"]))
display(Markdown(response))

TypeError: Markdown expects text, not AIMessage(content="Below is a table in markdown format summarizing all the shirts with sun protection:\n\n```markdown\n| Name                          | Description Summary                                                                                                                                     |\n|-------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------|\n| Sun Shield Shirt by           | A high-performance, slightly fitted sun shirt made of 78% nylon and 22% Lycra Xtra Life fiber. Offers UPF 50+ protection, moisture-wicking, and abrasion resistance. Handwash recommended. |\n| Men's TropicVibe Shirt        | A relaxed-fit short-sleeve shirt with UPF 50+ sun protection. Made from a lightweight nylon-polyester blend, it features wrinkle-resistant fabric, front and back cape venting, and two bellows pockets. Machine 

In [27]:
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

In [28]:
query =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."

In [29]:
response = qa_stuff.invoke(query)
# response = qa_stuff.run(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [30]:
print(response)

{'query': 'Please list all your shirts with sun protection in a table in markdown and summarize each one.', 'result': "| ID | Name | Summary |\n|----|------|---------|\n| 255 | Sun Shield Shirt | A high-performance sun shirt with a slightly fitted cut that falls at the hip. It's made of 78% nylon and 22% Lycra Xtra Life fiber, offering UPF 50+ protection. Handwash recommended. Features moisture-wicking technology and is abrasion-resistant. Blocks 98% of UV rays. |\n| 535 | Men's TropicVibe Shirt, Short-Sleeve | A men's short-sleeve shirt with traditional fit, relaxed through chest, sleeve, and waist. Made of 71% Nylon and 29% Polyester shell with polyester mesh lining, it offers UPF 50+ protection. Machine washable. Features wrinkle resistance, front and back cape venting, and two front bellows pockets. Blocks 98% of UV rays. |\n| 618 | Men's Tropical Plaid Short-Sleeve Shirt | A lightweight men's shirt with a traditional fit, relaxed through chest, sleeve, and waist. Made of 100% poly

In [31]:
type(response)

dict

In [32]:
display(Markdown(response["query"]))

Please list all your shirts with sun protection in a table in markdown and summarize each one.

In [33]:
display(Markdown(response["result"]))

| ID | Name | Summary |
|----|------|---------|
| 255 | Sun Shield Shirt | A high-performance sun shirt with a slightly fitted cut that falls at the hip. It's made of 78% nylon and 22% Lycra Xtra Life fiber, offering UPF 50+ protection. Handwash recommended. Features moisture-wicking technology and is abrasion-resistant. Blocks 98% of UV rays. |
| 535 | Men's TropicVibe Shirt, Short-Sleeve | A men's short-sleeve shirt with traditional fit, relaxed through chest, sleeve, and waist. Made of 71% Nylon and 29% Polyester shell with polyester mesh lining, it offers UPF 50+ protection. Machine washable. Features wrinkle resistance, front and back cape venting, and two front bellows pockets. Blocks 98% of UV rays. |
| 618 | Men's Tropical Plaid Short-Sleeve Shirt | A lightweight men's shirt with a traditional fit, relaxed through chest, sleeve, and waist. Made of 100% polyester, it provides UPF 50+ protection. Wrinkle-resistant with front and back cape venting and two front bellows pockets. Blocks 98% of UV rays. |
| 293 | Girls' Ocean Breeze Long-Sleeve Stripe Shirt | A long-sleeve rash guard for girls designed for full-coverage during beach or pool activities. Made of nylon and Lycra-elastane blend with UPF 50+ protection. Machine washable, quick-drying, fade-resistant, and seawater-resistant. Coordinates with swimsuits. Blocks 98% of UV rays. | 

### Summary:
- **Sun Shield Shirt**: A versatile, slightly fitted sun protection shirt with UPF 50+, moisture-wicking, and abrasion-resistant features.
- **Men's TropicVibe Shirt, Short-Sleeve**: A relaxed fit men's shirt offering UPF 50+ protection with cooling vents and wrinkle resistance.
- **Men's Tropical Plaid Short-Sleeve Shirt**: A lightweight plaid shirt with UPF 50+ protection, featuring cooling vents and a relaxed fit.
- **Girls' Ocean Breeze Long-Sleeve Stripe Shirt**: A durable, long-sleeve rash guard for girls with UPF 50+ protection, perfect for water activities.

In [34]:
response = index.query(query, llm=llm)

In [59]:
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings,
).from_loaders([loader])

In [36]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown

from langchain.indexes import VectorstoreIndexCreator
from langchain_community.embeddings.dashscope import DashScopeEmbeddings

In [37]:


# 加载文档
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
# embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')
index = VectorstoreIndexCreator(
    vectorstore_cls=DocArrayInMemorySearch,
    embedding=embeddings
).from_loaders([loader])
retriever = index.vectorstore.as_retriever()

# 初始化大语言模型
# llm = OpenAI(openai_api_key="your_api_key")
llm = ChatOpenAI(
    openai_api_base="https://dashscope.aliyuncs.com/compatible-mode/v1",
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    model_name="qwen-plus",	# 模型名称
    temperature=0.9
)


In [38]:

# 使用 stuff 类型
qa_stuff = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    verbose=True
)

# 使用 map_reduce 类型
qa_map_reduce = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_reduce", 
    retriever=retriever, 
    verbose=True
)

# 使用 refine 类型
qa_refine = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="refine", 
    retriever=retriever, 
    verbose=True
)

# 使用 map_rerank 类型
qa_map_rerank = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_rerank", 
    retriever=retriever, 
    verbose=True
)

question =  "Please list all your shirts with sun protection in a table \
in markdown and summarize each one."
# question = "你的问题是什么"
answer_stuff = qa_stuff.run(question)
answer_map_reduce = qa_map_reduce.run(question)
answer_refine = qa_refine.run(question)
answer_map_rerank = qa_map_rerank.run(question)

print("Stuff 类型答案:", answer_stuff)
print("Map Reduce 类型答案:", answer_map_reduce)
print("Refine 类型答案:", answer_refine)
print("Map Rerank 类型答案:", answer_map_rerank)

/var/folders/2p/9b5w6pj56_s1vvr_y6r4cmqc0000gn/T/ipykernel_26983/2222093920.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer_stuff = qa_stuff.run(question)




> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


/Users/zhaoyang/opt/anaconda3/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:174: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")



> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...


/Users/zhaoyang/opt/anaconda3/lib/python3.9/site-packages/langchain/chains/llm.py:369: UserWarning: The apply_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(



> Finished chain.
Stuff 类型答案: Below is a table summarizing all the shirts with sun protection:

| ID   | Name                                         | Material Composition                          | Features                                                                                                           | Summary                                                                                                                                                                                                                  |
|------|----------------------------------------------|---------------------------------------------|-------------------------------------------------------------------------------------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 255  | Sun Shield Shirt 

/Users/zhaoyang/opt/anaconda3/lib/python3.9/site-packages/langchain_community/chat_models/openai.py:174: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")
